Next, we need to set API keys for OpenAI (the LLM we will use) and Tavily (the search tool we will use)

In [19]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os
load_dotenv()

os.getenv("GROQ_API_KEY")
os.getenv("TAVILY_API_KEY")



<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>

## Define Tools

We will first define the tools we want to use. For this simple example, we will use a built-in search tool via Tavily. However, it is really easy to create your own tools - see documentation [here](https://python.langchain.com/docs/how_to/custom_tools) on how to do that.

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=3)]

## Define our Execution Agent

Now we will create the execution agent we want to use to execute tasks. 
Note that for this example, we will be using the same execution agent for each task, but this doesn't HAVE to be the case.

In [8]:
from langchain import hub

from langgraph.prebuilt import create_react_agent

# Get the prompt to use - you can modify this!
prompt = hub.pull("ih/ih-react-agent-executor")
prompt.pretty_print()

# Choose the LLM that will drive the agent

agent_executor = create_react_agent(llm, tools, state_modifier=prompt)

c:\Sagar\SOFTWARE\Anaconda\anaconda3\envs\fastapi\Lib\site-packages\langsmith\client.py:322: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  


================================ System Message ================================

You are a helpful assistant.

============================= Messages Placeholder =============================

{messages}


c:\Sagar\SOFTWARE\Anaconda\anaconda3\envs\fastapi\Lib\site-packages\langsmith\client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  "sort_field": sort_field,


In [9]:
agent_executor.invoke({"messages": [("user", "who is the winnner of the us open")]})

{'messages': [HumanMessage(content='who is the winnner of the us open', additional_kwargs={}, response_metadata={}, id='4b02af2f-aac1-4acb-918f-4ca696f40c45'),
  AIMessage(content="The winner of the US Open changes every year, so I need the specific year to provide the correct information. As of my last update, the men's singles winner of the 2021 US Open is Daniil Medvedev and the women's singles winner is Emma Raducanu.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 1226, 'total_tokens': 1291, 'completion_time': 0.103828977, 'prompt_time': 0.0614389, 'queue_time': 0.0008516930000000006, 'total_time': 0.165267877}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-5797b1c0-c56e-45fc-b4cc-74b62bbf9431-0', usage_metadata={'input_tokens': 1226, 'output_tokens': 65, 'total_tokens': 1291})]}

## Define the State

Let's now start by defining the state the track for this agent.

First, we will need to track the current plan. Let's represent that as a list of strings.

Next, we should track previously executed steps. Let's represent that as a list of tuples (these tuples will contain the step and then the result)

Finally, we need to have some state to represent the final response as well as the original input.

In [9]:
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict


class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

## Planning Step

Let's now think about creating the planning step. This will use function calling to create a plan.

<div class="admonition note">
    <p class="admonition-title">Using Pydantic with LangChain</p>
    <p>
        This notebook uses Pydantic v2 <code>BaseModel</code>, which requires <code>langchain-core >= 0.3</code>. Using <code>langchain-core < 0.3</code> will result in errors due to mixing of Pydantic v1 and v2 <code>BaseModels</code>.
    </p>
</div>

In [10]:
from pydantic import BaseModel, Field


class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

Plan(steps=['Find the current Australia Open winner', 'Find the hometown of the current Australia Open winner'])

In [3]:
tools1=[
  {
      "tool_name": "who_am_i",
      "tool_description": "Returns the id of the current user",
      "args": [],
      "output": {
          "arg_type": "str",
          "is_array": False,
          "is_required": True
      }
  },
  {
      "tool_name": "get_sprint_id",
      "tool_description": "Returns the ID\nof the current\nsprint",
      "args": [],
      "output": {
          "arg_type": "str",
          "is_array": False,
          "is_required": True
      }
  },
  {
    "tool_name": "works_list",
    "tool_description": "Returns a list of work items matching the request",
    "args": [
    {
    "arg_name": "applies_to_part",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "created_by",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "issue_priority",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "issue.rev_orgs",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "limit",
    "arg_type": "int",
    "is_array": False,
    "is_required": False
    },
    {
    "arg_name": "owned_by",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "stage_name",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "ticket_need_response",
    "arg_type": "boolean",
    "is_array": False,
    "is_required": False
    },
    {
    "arg_name": "ticket_rev_org",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "ticket_severity",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "ticket_source_channel",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "type",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    }
    ],
    "output": {
    "arg_type": "any",
    "is_array": False,
    "is_required": True
    }
    },
    {
    "tool_name": "summarize_objects",
    "tool_description": "Summarizes a list of objects. The logic of how to summarize a particular object type is an internal implementation detail.",
    "args": [
    {
    "arg_name": "objects",
    "arg_type": "any",
    "is_array": True,
    "is_required": True
    }
    ],
    "output": {
    "arg_type": "any",
    "is_array": False,
    "is_required": True
    }
    },
    {
    "tool_name": "prioritize_objects",
    "tool_description": "Returns a list of objects sorted by priority. The logic of what constitutes priority for a given object is an internal implementation detail",
    "args": [
    {
    "arg_name": "objects",
    "arg_type": "any",
    "is_array": False,
    "is_required": False
    }
    ],
    "output": {
    "arg_type": "any",
    "is_array": True,
    "is_required": True
    }
}]


In [23]:
llm = ChatGroq(
    model="llama3-groq-8b-8192-tool-use-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [24]:
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate


system_prompt="""For the given objective, come up with a simple step by step plan according the following tools provided 
{tools} """
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)
user_message_template = HumanMessagePromptTemplate.from_template("{user_query}")
planner_prompt = ChatPromptTemplate.from_messages([system_message_template, user_message_template])

planner = planner_prompt | llm.with_structured_output(Plan)

## calling tools according to plan



In [28]:
from typing import Union


class Argument(BaseModel):
    argument_name: str
    argument_value: List[str]

class Tool(BaseModel):
    tool_name: str
    arguments: List[Argument]
    
class ResponseModel(BaseModel):
    tool_calls: list[Tool]

def tool_calling(query,tools):
    plan=planner.invoke({"user_query":query,"tools":tools})
    print(plan)
    tool_prompt = ChatPromptTemplate.from_template(
    """For the given Query:{plan}, come up with  tool calling josn ITS NOT NECESAARY TO CALL TOOLS FOR EVERY STEPS. SOME TOOL MIGHT ACCOMPLISH MULTIPE STEOS TOGEATHER.understand the essence of te plan \
        You have access to the following tool: {tools}.go through each argument provided properly and ive every needed arg and select the relavent tools
        when you need output from of i th previous tool for argument write "$$PREV[i]" in the argument .order the function calling properly for the task .DONT CALL FUNCTION UNECESARRLY unless properly specified for""")
    
    tool_caller = tool_prompt | llm.with_structured_output(ResponseModel)
    print(tool_caller.invoke({"plan":plan,"tools":tools})) 

query="list their high-severity tickets for ’Globex’ user"


tool_calling(query,tools1)


steps=['Get user ID for Globex', 'Get current sprint ID', 'Fetch high-severity tickets for Globex in current sprint']


ValidationError: 8 validation errors for ResponseModel
tool_calls.2.arguments.0.argument_name
  Field required [type=missing, input_value={'applies_to_part': ['Glo...sue_priority': ['high']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
tool_calls.2.arguments.0.argument_value
  Field required [type=missing, input_value={'applies_to_part': ['Glo...sue_priority': ['high']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
tool_calls.2.arguments.1.argument_name
  Field required [type=missing, input_value={'created_by': ['$$PREV[0...ket_severity': ['high']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
tool_calls.2.arguments.1.argument_value
  Field required [type=missing, input_value={'created_by': ['$$PREV[0...ket_severity': ['high']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
tool_calls.2.arguments.